# Les fonctions

Il est possible de définir une fonction de trois façons différentes :

## Définition en ligne

In [1]:
f(x) = 3x + 1

f (generic function with 1 method)

In [2]:
f(2)

7

In [3]:
g(x,y)=[x*y+1, x-y]

g (generic function with 1 method)

In [4]:
g(1,2)

2-element Vector{Int64}:
  3
 -1

Il est possible d'utiliser **begin**-**end** ou **( ; )** pour délimiter la fonction la dernière valeur calculée est retournée

In [5]:
h(x) = begin
    y = 2
    x + y
end

h (generic function with 1 method)

In [6]:
h(3)

5

In [7]:
h(x) = (y=2; x+y) # equivallent à la première écriture

h (generic function with 1 method)

In [8]:
h(3)

5

## Structurée

*Julia* possède une structure plus classique à l'aide de **function**-**end** comme précédemment la dernière valeur calculée est par défaut la variable de retour autrement l'utilisation de **return** spécifie la ou les variables de sortie. 

In [10]:
function H(x,y)
    z = x+y
    return z^2/(abs(x)+1)
end

H (generic function with 1 method)

In [11]:
H(1,2)

4.5

L'usage de **return** pour fixer la sortie 

In [12]:
function Choix(x)
    if x>0
        return "Positif"
    else
        return "Négatif"
    end
end

Choix (generic function with 1 method)

In [13]:
txt=Choix(3)

"Positif"

## Anonyme

Le concept peut paraître abstrait mais on peut définir une fonction sans la nommer puis l'affecter à une variable...

In [14]:
x -> x^2

#1 (generic function with 1 method)

In [15]:
G = x -> sqrt(x)

#3 (generic function with 1 method)

In [16]:
G(1)

1.0

In [17]:
typeof(G)

var"#3#4"

## Arguments de sortie

Pour avoir plusieurs arguments de sortie il faut utiliser un "tuple" autrement dit une collection d'éléments

In [18]:
function multi_output(x,y)
    return x+y, x-y
end

multi_output (generic function with 1 method)

In [20]:
a = multi_output(1,2) # un seul argument de sortie qui contient un tuple

(3, -1)

In [21]:
typeof(a)

Tuple{Int64, Int64}

In [22]:
a[1]

3

In [23]:
a[2]

-1

In [25]:
a, b = multi_output(1,2) # assignation aux 2 arguments de sortie

(3, -1)

In [26]:
a

3

In [27]:
b

-1

## Portée des variables 

Quelle est la portée des variables, autrement dit une variable définie peut elle être accessible, modifiable dans une fonction sans la passer en paramètre ?

In [39]:
a = 1
f(x) = a + x
f(1)

LoadError: cannot declare a constant; it already has a value

In [67]:
const α = 1
function ff(x)
    x + α
end

@code_llvm ff(1)

;  @ In[67]:2 within `ff`
define i64 @julia_ff_2525(i64 signext %0) #0 {
top:
;  @ In[67]:3 within `ff`
; ┌ @ int.jl:87 within `+`
   %1 = add i64 %0, 1
; └
  ret i64 %1
}


In [56]:
a = 1
function ff(x)
    x + a # on utilise a défini en dehors de la fonction
    a = 2 # on tente de changer la valeur de a... error !
end
#ff(1)

ff (generic function with 1 method)

In [57]:
ff(1)

LoadError: UndefVarError: a not defined

In [35]:
a = 1
function ff(x)
    a = 2
    x + a
end
ff(1)

3

In [37]:
@code_lowered ff(1)

CodeInfo(
1 ─      a = 2
│   %2 = x + a
└──      return %2
)

In [38]:
a

1

Donc par défaut une variable définie est connue et utilisable par toute fonction appelée (de même à l'intérieur d'une fonction).

Si on redéfinit localement dans la fonction la variable alors "elle écrase localement" la dite variable et en sortie de fonction rien n'est modifié.

Attention à l'utilisation dans la fonction d'une variable extérieure puis d'affecter une valeur à cette variable...



## Le mapping

Souvent on écrit une fonction sous une forme scalaire sans chercher à utiliser les opérations vectorielles pour appliquer cette fonction à un tableau

In [40]:
f(x) = x^2 + 1
f(1:5)  # il aurait fallu définir f(x)=x.^2.+1

LoadError: MethodError: no method matching ^(::UnitRange{Int64}, ::Int64)
[0mClosest candidates are:
[0m  ^([91m::Union{AbstractChar, AbstractString}[39m, ::Integer) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/strings/basic.jl:721
[0m  ^([91m::Rational[39m, ::Integer) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/rational.jl:475
[0m  ^([91m::Complex{<:AbstractFloat}[39m, ::Integer) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/complex.jl:839
[0m  ...

La fonction <code>map</code> permet de palier à ce manquement

In [41]:
map(f,1:5)

5-element Vector{Int64}:
  2
  5
 10
 17
 26

In [42]:
f.(1:5)

5-element Vector{Int64}:
  2
  5
 10
 17
 26

In [43]:
v = [1 2; 3 4]
map(f,v)

2×2 Matrix{Int64}:
  2   5
 10  17

In [44]:
reduce(+, v )

10

In [45]:
sum(v)

10

In [46]:
f.([1 2; 3 4])

2×2 Matrix{Int64}:
  2   5
 10  17

In [47]:
g(x,y)=x+y
map(g,0:3,1:4)

4-element Vector{Int64}:
 1
 3
 5
 7

In [48]:
map(g,[1 2;3 4],[2 3;4 5])

2×2 Matrix{Int64}:
 3  5
 7  9

In [49]:
g(f,x)=f(x)+1

g (generic function with 1 method)

In [50]:
g(sin,1)

1.8414709848078965

Il existe également un opérateur pour "composer" les fonctions `\circ`

In [52]:
f1(x) = 2x + 1
f2(x) = 1 - 4x
(f1 ∘ f2)(3)

-21

In [53]:
f1(f2(3))

-21